# Continuous Control

---

This notebook contains my solution to the Continuos control problem


## 1. Create Environment and Observe State and Action spaces

The following section cretaes the unity environment.

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name='Reacher.app')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


## 2. Create the Agent
The following section creates the agent and makes more necessary imports.

In [2]:
# Make all important imorts and create agent

import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent


agent = Agent(state_size=33, action_size=4, random_seed = 0)

## 3. (Optional) Training the Agent using a DDPG Algorithm
The next section trains the agent using a ddpg algoritm until the target reward is achived.
The target reward is achived if the average reward over the last 100 episodes is above 30.


In [ ]:
def ddpg(n_episodes: int =  1500, max_actions:int = 400 ):
    scores = []
    scores_window = deque(maxlen=100)
    
    
    for i_episode in range(1,n_episodes+1):
        
        #reset the environment for every episode and initialize the state
        #step_brain_info = env.reset(train_mode=i_episode % 100 != 0)[brain_name] # reset the environment
        step_brain_info = env.reset(train_mode=True)[brain_name] # reset the environment
        
        
        state = step_brain_info.vector_observations[0]   #obtain the starting state
        
        score = 0 #initialze the score for the episode
        
        
        for a in range(max_actions):
            action = agent.act(state)
            
            step_brain_info = env.step(action)[brain_name]
            
            next_state: np.ndarray = step_brain_info.vector_observations[0]   # get the next state
            reward = step_brain_info.rewards[0]                   # get the reward
            done = step_brain_info.local_done[0]                  # see if episode has finished
            
            score += reward
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state                             # roll over the state to next time step
            if done:                             # exit loop if episode finished
                break
                
            if env.global_done:
                print('Global Done Steps: '+ str(a))
                break
        
        scores.append(score)
        scores_window.append(score)       # save most recent score
        
        print('\rEpisode {}\tavg Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tavg Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            #torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
            
    
    return scores
            
    
scores =  ddpg()
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show() 
    
    

Episode 100	avg Score: 0.87
Episode 104	avg Score: 0.96

In [8]:
#Save the learned weights
agent.save_weights()

# 4.  Observe the agent in the environment

In [4]:
# Observe the agent in its enviroment
agent.restore_weights()
n_episodes = 2
for i_episode in range(1,n_episodes+1):
    
    print('\rEpisode ' + str(i_episode), end="")
    #reset the environment for every episode and initialize the state
    #step_brain_info = env.reset(train_mode=i_episode % 100 != 0)[brain_name] # reset the environment
    step_brain_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = step_brain_info.vector_observations[0]   #obtain the starting state
    
    for a in range(500):
        action = agent.act(state)
            
        step_brain_info = env.step(action)[brain_name]
            
        next_state: np.ndarray = step_brain_info.vector_observations[0]   # get the next state
        reward = step_brain_info.rewards[0]                   # get the reward
        done = step_brain_info.local_done[0]                  # see if episode has finished
             
        state = next_state                             # roll over the state to next time step
        if done:                             # exit loop if episode finished
            break
                
        if env.global_done:
            print('Global Done Steps: '+ str(a))
            break
    
    
 

Episode 500

In [ ]:
# See the trained agent in action

